# Logic Creation and Experimentation Notebook:

In [1]:
# Import statements:
import pandas as pd

In [2]:
# Global variables
STUDENT_INFO_FILEPATH = "./data/student_information.csv" # path to the ".csv" file with the student information.
REQ_COLS = ["student_id", "student_name", "student_grade"] # list of columns within the file.

In [5]:
# User defined function to read and validate the student information file:
def read_student_info(student_info_filepath, req_cols):
    """
        Reads a ".csv" file with the student information, checks if the required
        columns exist in the file and returns the file as a pandas DataFrame.
        
        Arguments:
            1. student_info_filepath - str - Path to the ".csv" file with the student information.
            2. req_cols - list - List of str values representing the mandatory columns form the
                student information file.

        Returns:
            1. student_info_df - pandas.core.DataFrame - The student information as a pandas DataFrame.
    """
    # Reading the student information file:
    student_info_df = pd.read_csv(student_info_filepath)
    # Standardisation of the column names:
    student_info_file_cols = [
        f.lower().strip().replace(" ", "_") for f in student_info_df.columns
    ]
    # Checking for missing columns in the files:
    missing_cols = set(req_cols) - set(student_info_file_cols)
    if len(missing_cols) == 0: # If no columns are missing.
        return student_info_df
    else: # If at-least one column is missing.
        raise ValueError(f"The columns {missing_cols} are missing.")

In [4]:
# Reading the student file:
student_info_df = read_student_info(
    student_info_filepath=STUDENT_INFO_FILEPATH, req_cols=REQ_COLS
)